In [1]:
# load and process data
# separate into features and labels
# train test split
# scale / label
# datasets and dataloaders
# define model
# parameters and supporting functions
# training loop
# evaluation loop

In [1]:
import pandas as pd
import numpy as np
import torch

In [3]:
torch.manual_seed(13)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
df = pd.read_csv('./data/fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [6]:
x = df.iloc[:, 1:].to_numpy()
y = df.iloc[:, 0].to_numpy()

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [8]:
x_train_scaled = x_train / 255.0
x_test_scaled = x_test / 255.0

In [9]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        super().__init__()
        self.features = torch.tensor(features, dtype = torch.float32)
        self.labels = torch.tensor(labels, dtype = torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]
    

In [10]:
trainds = CustomDataset(x_train_scaled, y_train)
testds = CustomDataset(x_test_scaled, y_test)

In [11]:
import torch.nn as nn

# Optimization : ----------------------------
# add a dropout layer after activation 
# add a batch normalization before activation
# add regularization ( add weight decay to optimizer step)
class MyOAnn(nn.Module):
    def __init__(self, num_hidden_layers, neurons_per_layer,
                input_dim, output_dim, dropout_rate):
        super().__init__()
        layers = []
        layers.append(nn.Linear(input_dim, neurons_per_layer))
        layers.append(nn.BatchNorm1d(neurons_per_layer))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout1d(p=dropout_rate))
        
        for i in range(num_hidden_layers):
            layers.append(nn.Linear(neurons_per_layer, neurons_per_layer))
            layers.append(nn.BatchNorm1d(neurons_per_layer))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout1d(p=dropout_rate))

        layers.append(nn.Linear(neurons_per_layer, output_dim))

        self.model = nn.Sequential(*layers)

    def forward(self, X):
        return self.model(X)
        

In [12]:
def objective(trial):
    # get hyperparameter values
    num_hidden_layers = trial.suggest_int('num_hidden_layer', 1, 5)
    num_neurons_per_layer = trial.suggest_int('neurons_per_layer', 8, 128, step = 8)
    epochs = trial.suggest_int("epochs", 10, 50, step = 10)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32, 64])
    optimizer_model = trial.suggest_categorical('optimizer_model', ['sgd', 'adam', 'rmsprop'])
    
    # model init
    input_dim = 784
    output_dim = 10

    model = MyOAnn(input_dim = input_dim, output_dim=output_dim, 
                   num_hidden_layers= num_hidden_layers, neurons_per_layer=num_neurons_per_layer,
                  dropout_rate=dropout_rate)

    model.to(device)
        
    # select optimizer and loss function
    loss_function = nn.CrossEntropyLoss()
    
    if optimizer_model == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay)
    elif optimizer_model == 'rmsprop':
        optimizer = torch.optim.RMSprop(model.parameters(), lr = learning_rate, weight_decay=weight_decay)
    else:
        optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, weight_decay=weight_decay)

    # loaders
    train_loader = DataLoader(trainds, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(testds, batch_size=batch_size, shuffle=False, pin_memory=True)
    
    # training loop
    for epoch in range(epochs):
    
        total_epoch_loss = 0
        
        for bfeatures, blabels in train_loader:
            bfeatures, blabels = bfeatures.to(device), blabels.to(device)
            out = model(bfeatures)
            loss = loss_function(out, blabels)
    
            optimizer.zero_grad()
            loss.backward()
    
            optimizer.step()
    
            total_epoch_loss += loss.item()

        avg_loss = total_epoch_loss / len(train_loader)
        print(f'Epoch: {epoch + 1} | loss: {avg_loss}')
        
    
    # evaluation loop
    model.eval()
    
    total = 0
    correct = 0

    with torch.no_grad():
        for bfeatures, blabels in test_loader:
            bfeatures, blabels = bfeatures.to(device), blabels.to(device)
            out = model(bfeatures)
            _, predicted = torch.max(out, 1)
            total += blabels.shape[0]
            correct += (predicted == blabels).sum().item()
    
    accuracy = correct/ total


    return accuracy

In [13]:
import optuna
study = optuna.create_study(direction='maximize')

VersionError: Detected incompatible Protobuf Gencode/Runtime versions when loading api.proto: gencode 5.28.1 runtime 5.27.5. Runtime version cannot be older than the linked gencode version. See Protobuf version guarantees at https://protobuf.dev/support/cross-version-runtime-guarantee.

In [30]:
study.optimize(objective, n_trials=100)

Epoch: 1 | loss: 1.5200910433133443
Epoch: 2 | loss: 1.3638951281706493
Epoch: 3 | loss: 1.3409531585375467
Epoch: 4 | loss: 1.2723916888237
Epoch: 5 | loss: 1.2570390661557516
Epoch: 6 | loss: 1.2473220149676005
Epoch: 7 | loss: 1.2086151909828187
Epoch: 8 | loss: 1.2161643731594085
Epoch: 9 | loss: 1.2196636041005453
Epoch: 10 | loss: 1.1886268305778502
Epoch: 11 | loss: 1.1740487082799276
Epoch: 12 | loss: 1.1421062358220417
Epoch: 13 | loss: 1.1323706591129303
Epoch: 14 | loss: 1.1472466226418814
Epoch: 15 | loss: 1.1080278944969177
Epoch: 16 | loss: 1.1304423801104229
Epoch: 17 | loss: 1.1220671566327414
Epoch: 18 | loss: 1.126305454572042
Epoch: 19 | loss: 1.1221128459771474
Epoch: 20 | loss: 1.0785150599479676
Epoch: 21 | loss: 1.118918940226237
Epoch: 22 | loss: 1.0764168246587118
Epoch: 23 | loss: 1.0585569043954213
Epoch: 24 | loss: 1.058400828440984
Epoch: 25 | loss: 1.0473980379104615
Epoch: 26 | loss: 1.0340876770019531
Epoch: 27 | loss: 1.078738973736763
Epoch: 28 | loss:

[I 2025-02-02 10:23:20,738] Trial 0 finished with value: 0.8525 and parameters: {'num_hidden_layer': 4, 'neurons_per_layer': 56, 'epochs': 30, 'learning_rate': 0.0035102428763286587, 'dropout_rate': 0.1, 'weight_decay': 1.1418112583335027e-05, 'batch_size': 32, 'optimizer_model': 'rmsprop'}. Best is trial 0 with value: 0.8525.


Epoch: 30 | loss: 1.050108722448349
Epoch: 1 | loss: 2.378549378712972
Epoch: 2 | loss: 2.327552499771118
Epoch: 3 | loss: 2.3114326922098796
Epoch: 4 | loss: 2.284178245862325
Epoch: 5 | loss: 2.27047176361084
Epoch: 6 | loss: 2.264711283047994
Epoch: 7 | loss: 2.2539017740885416
Epoch: 8 | loss: 2.250886007944743
Epoch: 9 | loss: 2.2513408438364664
Epoch: 10 | loss: 2.2432899777094524
Epoch: 11 | loss: 2.232193916638692
Epoch: 12 | loss: 2.236861055692037
Epoch: 13 | loss: 2.2382079267501833
Epoch: 14 | loss: 2.2379424079259236
Epoch: 15 | loss: 2.2177187760670978
Epoch: 16 | loss: 2.2207020203272503
Epoch: 17 | loss: 2.2183447535832723
Epoch: 18 | loss: 2.227098337809245
Epoch: 19 | loss: 2.215122776826223
Epoch: 20 | loss: 2.21902778784434
Epoch: 21 | loss: 2.2086700359980265
Epoch: 22 | loss: 2.210611956914266
Epoch: 23 | loss: 2.1985168631871543
Epoch: 24 | loss: 2.207915121714274
Epoch: 25 | loss: 2.2162954211235046
Epoch: 26 | loss: 2.218963950475057
Epoch: 27 | loss: 2.2063472

[I 2025-02-02 10:23:33,719] Trial 1 finished with value: 0.5 and parameters: {'num_hidden_layer': 4, 'neurons_per_layer': 64, 'epochs': 30, 'learning_rate': 0.0007635250112192188, 'dropout_rate': 0.4, 'weight_decay': 6.596405298297512e-05, 'batch_size': 32, 'optimizer_model': 'sgd'}. Best is trial 0 with value: 0.8525.


Epoch: 30 | loss: 2.196143612066905
Epoch: 1 | loss: 2.312933389345805
Epoch: 2 | loss: 2.2272514979044598
Epoch: 3 | loss: 2.2049594148000082
Epoch: 4 | loss: 2.2025992663701377
Epoch: 5 | loss: 2.1942679707209267
Epoch: 6 | loss: 2.192202161947886
Epoch: 7 | loss: 2.196279894510905
Epoch: 8 | loss: 2.1957363080978394
Epoch: 9 | loss: 2.1978750212987266
Epoch: 10 | loss: 2.196917040348053
Epoch: 11 | loss: 2.1844138964017232
Epoch: 12 | loss: 2.1865816831588747
Epoch: 13 | loss: 2.192880360285441
Epoch: 14 | loss: 2.1924711910883588
Epoch: 15 | loss: 2.183751129309336
Epoch: 16 | loss: 2.1696779600779217
Epoch: 17 | loss: 2.170250573952993
Epoch: 18 | loss: 2.176437550385793
Epoch: 19 | loss: 2.178978261152903
Epoch: 20 | loss: 2.1753541334470112
Epoch: 21 | loss: 2.190083928902944
Epoch: 22 | loss: 2.184953440030416
Epoch: 23 | loss: 2.1682394313812257
Epoch: 24 | loss: 2.191653470993042
Epoch: 25 | loss: 2.1782161688804624
Epoch: 26 | loss: 2.199293167591095
Epoch: 27 | loss: 2.1747

...

Epoch: 30 | loss: 0.9273763271793724
Epoch: 1 | loss: 1.9592527039845784
Epoch: 2 | loss: 1.8031070494651795
Epoch: 3 | loss: 1.7761820713678995
Epoch: 4 | loss: 1.7647580083211263
Epoch: 5 | loss: 1.7495707368850708
Epoch: 6 | loss: 1.7349288407961527
Epoch: 7 | loss: 1.7213879068692526
Epoch: 8 | loss: 1.723517770767212
Epoch: 9 | loss: 1.7125668422381084
Epoch: 10 | loss: 1.6879627776145936
Epoch: 11 | loss: 1.6844833993911743
Epoch: 12 | loss: 1.6503143223126728
Epoch: 13 | loss: 1.66068568388621
Epoch: 14 | loss: 1.6937815594673156
Epoch: 15 | loss: 1.6792769130071004
Epoch: 16 | loss: 1.6714658665657043
Epoch: 17 | loss: 1.6810653797785442
Epoch: 18 | loss: 1.638935107390086
Epoch: 19 | loss: 1.6647084045410157
Epoch: 20 | loss: 1.6572761901219686
Epoch: 21 | loss: 1.6655431246757508
Epoch: 22 | loss: 1.6540107941627502
Epoch: 23 | loss: 1.6536379686991374
Epoch: 24 | loss: 1.640973834991455
Epoch: 25 | loss: 1.662843816280365
Epoch: 26 | loss: 1.6099563320477803
Epoch: 27 | loss

[I 2025-02-02 10:58:52,571] Trial 98 finished with value: 0.8208333333333333 and parameters: {'num_hidden_layer': 4, 'neurons_per_layer': 40, 'epochs': 30, 'learning_rate': 0.0006829350279338706, 'dropout_rate': 0.2, 'weight_decay': 0.00012985886644861552, 'batch_size': 32, 'optimizer_model': 'rmsprop'}. Best is trial 46 with value: 0.8566666666666667.


Epoch: 30 | loss: 1.6487674164772033
Epoch: 1 | loss: 1.761941173672676
Epoch: 2 | loss: 1.4013185181220373
Epoch: 3 | loss: 1.288479832559824
Epoch: 4 | loss: 1.2302641040086746
Epoch: 5 | loss: 1.1831178716073434
Epoch: 6 | loss: 1.1458215541640917
Epoch: 7 | loss: 1.0841136965155602
Epoch: 8 | loss: 1.1001701168964306
Epoch: 9 | loss: 1.0569301223754883
Epoch: 10 | loss: 1.01922087199986
Epoch: 11 | loss: 1.00105522736907
Epoch: 12 | loss: 1.0084187338252861
Epoch: 13 | loss: 0.9833780900761485
Epoch: 14 | loss: 0.9918364117170374
Epoch: 15 | loss: 0.9690644872188569
Epoch: 16 | loss: 0.9470104263722896
Epoch: 17 | loss: 0.9461946862687667
Epoch: 18 | loss: 0.9298730126023292
Epoch: 19 | loss: 0.920345288614432
Epoch: 20 | loss: 0.8846679895371199
Epoch: 21 | loss: 0.897920422355334
Epoch: 22 | loss: 0.8712201634794474
Epoch: 23 | loss: 0.8759100235005219
Epoch: 24 | loss: 0.879924930408597
Epoch: 25 | loss: 0.8624423550069332
Epoch: 26 | loss: 0.8803852870439489
Epoch: 27 | loss: 0

[I 2025-02-02 10:59:31,276] Trial 99 finished with value: 0.85 and parameters: {'num_hidden_layer': 3, 'neurons_per_layer': 104, 'epochs': 40, 'learning_rate': 0.00011705075073490327, 'dropout_rate': 0.1, 'weight_decay': 9.419316326123061e-05, 'batch_size': 8, 'optimizer_model': 'adam'}. Best is trial 46 with value: 0.8566666666666667.


Epoch: 40 | loss: 0.7966235018583635


In [ ]:
study.best_value